# Run Resnet50 Inference benchmarks on OpneVINO with synthetic data

### Write benchmark commands into a file benchmark_openvino-rn50.sh

In [11]:
%%writefile benchmark_openvino-rn50.sh
# activate opensource tf environment
source ~/opentfov/bin/activate
source  /opt/intel/openvino/bin/setupvars.sh

# get trained model from intel repo
cd ~/oneapi_workshop
wget https://download.01.org/opencv/public_models/012020/resnet-50-tf/resnet_v1-50.pb
#convert the tensorflow model to openVINO format
python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_shape=[1,224,224,3] --mean_values=[123.68,116.78,103.94] --input=map/TensorArrayStack/TensorArrayGatherV3 --output=softmax_tensor --input_model=$HOME/oneapi_workshop/resnet_v1-50.pb --reverse_input_channels
# run openVINO benchmarks
# run for batch size 1 and 1 stream and notedown FPS
$HOME/oneapi_workshop/samples/intel64/Release/benchmark_app -m $HOME/oneapi_workshop/resnet_v1-50.xml -nstreams 1 -niter 100 -b 1
# run for batch size 1 and 4 streams and notedown FPS
$HOME/oneapi_workshop/samples/intel64/Release/benchmark_app -m $HOME/oneapi_workshop/resnet_v1-50.xml -nstreams 4 -niter 100 -b 1
# run for batch size 16 and 1 stream and notedown FPS
$HOME/oneapi_workshop/samples/intel64/Release/benchmark_app -m $HOME/oneapi_workshop/resnet_v1-50.xml -nstreams 1 -niter 100 -b 16
# run for batch size 16 and 4 streams and notedown FPS
$HOME/oneapi_workshop/samples/intel64/Release/benchmark_app -m $HOME/oneapi_workshop/resnet_v1-50.xml -nstreams 4 -niter 100 -b 16
deactivate

Overwriting benchmark_openvino-rn50.sh


### Check content of benchmark_inteltf.sh

In [12]:
!cat benchmark_openvino-rn50.sh

# activate opensource tf environment
source ~/opentfov/bin/activate
source  /opt/intel/openvino/bin/setupvars.sh

# get trained model from intel repo
cd ~/oneapi_workshop
wget https://download.01.org/opencv/public_models/012020/resnet-50-tf/resnet_v1-50.pb
#convert the tensorflow model to openVINO format
python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_shape=[1,224,224,3] --mean_values=[123.68,116.78,103.94] --input=map/TensorArrayStack/TensorArrayGatherV3 --output=softmax_tensor --input_model=$HOME/oneapi_workshop/resnet_v1-50.pb --reverse_input_channels
# run openVINO benchmarks
# run for batch size 1 and 1 stream and notedown FPS
$HOME/oneapi_workshop/samples/intel64/Release/benchmark_app -m $HOME/oneapi_workshop/resnet_v1-50.xml -nstreams 1 -niter 100 -b 1
# run for batch size 1 and 4 streams and notedown FPS
$HOME/oneapi_workshop/samples/intel64/Release/benchmark_app -m $HOME/oneapi_workshop/resnet_v1-50.xml -nstreams 4 -niter 100 -b 1
# run for batch 

### Optional step : Remove all old output files

In [13]:
!rm -rf benchmark_openvino-rn50.sh.*

### Submit to queue.

In [14]:
!qsub benchmark_openvino-rn50.sh -l nodes=1:ppn=2 -d .

754105.v-qsvr-1.aidevcloud


### check job status

In [17]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
754062.v-qsvr-1            ...ub-singleuser u48334          00:00:51 R jupyterhub     


### If the job status is finished, check the the output file with proper output name.

In [18]:
!cat benchmark_openvino-rn50.sh.o*


########################################################################
#      Date:           Mon Dec  7 09:14:42 PST 2020
#    Job ID:           754105.v-qsvr-1.aidevcloud
#      User:           u48334
# Resources:           neednodes=1:ppn=2,nodes=1:ppn=2,walltime=06:00:00
########################################################################

[setupvars.sh] OpenVINO environment initialized
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u48334/oneapi_workshop/resnet_v1-50.pb
	- Path for generated IR: 	/home/u48334/oneapi_workshop/.
	- IR output name: 	resnet_v1-50
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	map/TensorArrayStack/TensorArrayGatherV3
	- Output layers: 	softmax_tensor
	- Input shapes: 	[1,224,224,3]
	- Mean values: 	[123.68,116.78,103.94]
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing

### Notedown the number of images/sec for training and inference

In [19]:
!grep -e "batch size" -e "inference requests" -e "Throughput"  benchmark_openvino-rn50.sh.o*

[ INFO ] Network batch size was changed to: 1
[Step 10/11] Measuring performance (Start inference asyncronously, 1 inference requests using 1 streams for CPU, limits: 100 iterations)
Throughput: 112.30 FPS
[ INFO ] Network batch size was changed to: 1
[Step 10/11] Measuring performance (Start inference asyncronously, 4 inference requests using 4 streams for CPU, limits: 100 iterations)
Throughput: 178.60 FPS
[ INFO ] Network batch size was changed to: 16
[Step 10/11] Measuring performance (Start inference asyncronously, 1 inference requests using 1 streams for CPU, limits: 100 iterations)
Throughput: 182.28 FPS
[ INFO ] Network batch size was changed to: 16
[Step 10/11] Measuring performance (Start inference asyncronously, 4 inference requests using 4 streams for CPU, limits: 100 iterations)
Throughput: 198.79 FPS


### close the notebook